# Transit duration

The maximum duration of a transit, $\Delta T$, for a single object on a near parabolic orbit is given by the equation:

$\Delta T \lesssim {\sqrt{2} R_*}/{\sqrt{q \times dv_r/dt}}$

We use the results from the 2018-03-17 data to calculate the transit duration.

In [64]:
import numpy as np
from astropy import units as u
from astropy import constants as const

def w_ave(x, xerr):
    weights = 1./(xerr**2)
    w_ave   = np.average(x, axis=0, weights=weights)
    uncertainty = 1/np.sum(weights)*np.sqrt(np.sum((weights*xerr)**2))
    return w_ave, uncertainty
    

In [65]:
dist        = np.array([9.0,7.1])
dist_err    = np.array([4.3,2.7])

dvdt        = np.array([1.8,2.9])
dvdt_err    = np.array([0.9,1.1])

dist_w_ave, dist_uncertainty = w_ave(dist, dist_err)
dvdt_w_ave, dvdt_uncertainty = w_ave(dvdt, dvdt_err)

print("Distance from star:",round(dist_w_ave,1),"±",round(dist_uncertainty,1),"R*")
print("Confidence:",round(dist_w_ave/dist_uncertainty,1),"sigma")

print("\nAcceleration:",round(dvdt_w_ave,1),"±",round(dvdt_uncertainty,1),"m/s^2")
print("Confidence:",round(dvdt_w_ave/dvdt_uncertainty,1),"sigma")

Distance from star: 7.6 ± 2.3 R*
Confidence: 3.3 sigma

Acceleration: 2.2 ± 0.7 m/s^2
Confidence: 3.2 sigma


In [66]:
R = 1.8*const.R_sun
q = dist_w_ave*R
dvdt = dvdt_w_ave*u.meter/u.second**2

In [67]:
dT = np.sqrt(2)*R/np.sqrt(q*dvdt)
dT_err = 1./(np.sqrt((dist_w_ave/dist_uncertainty))+np.sqrt(dvdt_w_ave/dvdt_uncertainty))
print("Maximum transit duration:",round(dT.to(u.hour).value,1),"±",round(dT_err,1),"hours")

Maximum transit duration: 3.4 ± 0.3 hours


A exocometary feature is first seen at 04:37:42 and lasting until the end of observations at 07:48:43. A difference of 3 h 11 min. We estimate that the transit duration should at most last 3 h 24 min. It is such a shame we were not able to observe for another 30 min to an hour. We could have caught the end of the transit perhaps!